## Exercises in Machine Learning

*Author*: Zach del Rosario

The _primary_ purpose of this notebook is to help you *not get fooled by machine learning*! As Drew Conway notes, possessing hacking skills and substantive experience---but having no math or statistics background---puts one in the [danger zone](http://drewconway.com/zia/2013/3/26/the-data-science-venn-diagram). While we can't possibly cover *everything* you need in a single workshop, this exercise will highlight some of the challenges of doing machine learning well. 

### Learning outcomes
By working through this notebook, you will be able to:

- Use grama to work with models.
- Understand the importance of *underfitting* and *overfitting*
- Use cross-validation to help avoid *underfitting* and *overfitting*

Tips:

* This exercise will make heavy use of [scikit-learn](https://scikit-learn.org/stable/); you can find lots of useful info on the [documentation site](https://scikit-learn.org/stable/documentation.html).


In [ ]:
# Setup
import numpy as np
import pandas as pd
import plotnine as pt
import grama as gr

DF = gr.Intention()

# Model training tools
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Helper functions
def add_noise(function, sigma=0.1, seed=101):
    """Add noise to deterministic functions
    """
    def new_function(x):
        np.random.seed(seed)
        y = function(x)
        return y + sigma * np.random.normal(size=y.shape)

    return new_function

# Reference points for regression examples
X_ref = np.linspace(-1, +1, num=100)

# Reference models
def fcn_1(x): return (0.3 * x**2 + 1.0 * x + 2)
def fcn_2(x): return (-2.0 * x**3 + 0.4 * x**2 + 1.0 * x + 2)

fcn_1_noisy = gr.make_symbolic(add_noise(fcn_1))
fcn_2_noisy = gr.make_symbolic(add_noise(fcn_2, sigma=0.2))

# Package as a dataframe
df_data = gr.df_make(
    x=X_ref,
    y_1=fcn_1_noisy(X_ref),
    y_2=fcn_2_noisy(X_ref),
)

## Helper Function

---

(This function will automate some steps so we can focus on high-level ideas, but I define it here in case you'd like to see the details)


In [ ]:
## For partial evaluation
from toolz import curry

## Encapsulate featurizer and regressor in one function
class FunctionLM(gr.Function):
    def __init__(self, regressor, var, out, name, order, runtime):
        self.regressor = regressor
        self.var = var
        self.out = list(map(lambda s: s + "_mean", out))
        self.name = name
        self.order = order
        self.runtime = runtime

    def eval(self, df):
        ## Check invariant; model inputs must be subset of df columns
        if not set(self.var).issubset(set(df.columns)):
            raise ValueError(
                "Model function `{}` var not a subset of given columns".format(
                    self.name
                )
            )

        ## Featurize
        x = np.atleast_2d(df[self.var].values)
        poly = PolynomialFeatures(self.order)
        X_feat = poly.fit_transform(x)
        
        ## Predict
        y = self.regressor.predict(X_feat).flatten()
        return pd.DataFrame(data=y, columns=self.out)

## Fitting routine
@curry
def fit_regression(df, var=None, out=None, order=1):
    r"""Fit a linear regression of specified order
    
    Args:
        df (DataFrame): Data for fitting
        var (iterable of str): Names of input variable (feature); must be column in df
        out (iterable of str): Name of output variable (response); must be column in df
        order (int): Polynomial order for fit
    
    """
    if len(out) > 1:
        raise ValueError("This simple helper")
    # Featurize
    x = np.atleast_2d(df[var].values)
    poly = PolynomialFeatures(order)
    X_feat = poly.fit_transform(x)
    
    # Fit regression
    y = np.atleast_2d(df[out].values)
    lm = LinearRegression()
    lm.fit(X_feat, y)
    
    # Package
    fun = FunctionLM(lm, var, out, "Linear Model", order, 0)
    
    return gr.Model(functions=[fun], domain=None, density=None)
    
## Create pipe-enabled regression utility
ft_regression = gr.add_pipe(fit_regression)


## Primer: The Key Ideas
First we'll cover some key ideas on simple functions. These are not 'real' data, but the simplicity of the examples will allow us to focus on concepts.

Here I generate some data from a simple polynomial.

In [ ]:
## NOTE: No need to edit
# Ground-truth data; no noise
df_ex1 = (
    gr.df_make(x=np.linspace(-1, +1, num=10))
    >> gr.tf_mutate(y=fcn_1(DF.x))
)

# Plot
(
    df_ex1
    >> pt.ggplot(pt.aes("x", "y"))
    + pt.geom_point()
)

We will fit a simple [linear regression](https://en.wikipedia.org/wiki/Linear_regression) to these data. To do this, we'll use the linear regression helper `ft_regression()` (defined above). To start, we'll assume that the data were generated from an underlying rule (a _model_) of the form

$$y_{\text{mean}} = m x + b,$$

and attempt to _learn_ the slope $m$ and intercept $b$ by _minimizing_ the difference between the measured values `y` and the predicted values `y_mean`.


In [ ]:
## NOTE: No need to edit
# Fit the line
md_line = (
    df_ex1
    >> ft_regression(var=["x"], out=["y"], order=1)
)

# Predict
df_line_pred = (
    md_line
    >> gr.ev_df(df=df_ex1)
)

# Plot results
(
    df_line_pred
    >> pt.ggplot(pt.aes("x", "y_mean"))
    + pt.geom_line(linetype="dashed", color="grey")
    + pt.geom_point(color="grey")
    + pt.geom_point(pt.aes(y="y"))
)

## Working with models in Grama


In [ ]:
## NOTE: No need to edit, run to show info on `md_line`
md_line


Based on this summary of the model, we can see that it takes a single input `x` and maps it to a single output `y_mean`.


### __QX__: Evaluate the model at new input values

Use the function `gr.ev_df()` to evaluate the model `md_line` fitted above. Make use of the new set of input values `df_dense`.


In [ ]:
###
# TASK: Evaluate md_line at the df_dense input values.
###

# -- FINISH THE CODE BELOW -----
df_dense = gr.df_make(x=np.linspace(-1, +1, num=100))

df_evaluated = (
    md_line
## TODO: Evaluate the model here

)

# -- NO NEED TO EDIT BELOW HERE -----
(
    df_evaluated
    >> pt.ggplot(pt.aes("x", "y_mean"))
    + pt.geom_line(linetype="dashed", color="grey")
    + pt.geom_point(color="grey")
    + pt.geom_point(
        data=df_ex1,
        mapping=pt.aes(y="y"),
    )
)

### Model Flexibility and _Underfitting_

Note that we had to _assume_ a model-form in order to do the fitting. From the figure above, we can see that the model is close to the true values, but obviously lacks the curvature of the true data-generating process.

This phenomenon---the failure of a model to capture behavior in the data---is called _underfitting_. To reduce underfitting, we need to make our model _more flexible_. For a linear regression, we can do this by **increasing the order** of the regression.


### __QX__: Fit a quadratic model to the data

Use the `ft_regression()` helper to fit a quadratic model (`order = 2`).


In [ ]:
###
# TASK: Fit a quadratic model
###

# -- FINISH THE CODE BELOW -----
# Fit a quadratic
md_quad = (
    df_ex1
## TODO: Fit a regression here

)

# -- NO NEED TO EDIT BELOW HERE -----
# Predict
df_quad_pred = (
    md_quad
    >> gr.ev_df(df=df_ex1)
)

# Plot results
(
    df_quad_pred
    >> pt.ggplot(pt.aes("x", "y_mean"))
    + pt.geom_line(linetype="dashed")
    + pt.geom_point(pt.aes(y="y"))
)

Here we can see the model fits the data _perfectly_. This _suggests_ that we have successfully discovered the _exact_ rule that generated these data, which in _this special case happens to be true_.

However, we will very rarely be able to fit the true function exactly. This is because real data tend to have _noise_, which corrupts the underlying function we are trying to learn.

### Noise and _Overfitting_
Below, I generate data from the same model, but add a little bit of noise.

In [ ]:
## NOTE: No need to edit
# Ground-truth data; now with noise
df_ex2 = (
    gr.df_make(x=np.linspace(-1, +1, num=10))
    >> gr.tf_mutate(y=fcn_1_noisy(DF.x))
)

# Plot
(
    df_ex2
    >> pt.ggplot(pt.aes("x", "y"))
    + pt.geom_point()
)

Let's see what happens when we fit a quadratic to the noisy data.


In [ ]:
## NOTE: No need to edit
(
    df_ex2
    >> ft_regression(var=["x"], out=["y"], order=2)
    >> gr.ev_df(df=df_ex2)
    
    >> pt.ggplot(pt.aes("x", "y_mean"))
    + pt.geom_line(linetype="dashed")
    + pt.geom_point(pt.aes(y="y"))
)

Here we can see that the fit is no longer perfect, despite coming from the "same" model. Since we already know that a quadratic can fit the underlying function perfectly, underfitting is not the issue here. Instead, the error is increased due to the noise in the data.

_However_, we have not yet seen a case of _overfitting_. To see that phenomenon, let's keep increasing the order of the model.

### __QX__: Overfit the model

Increase the `order` of the regression until the model goes through every measured point. Does this seem like a reasonable model?


In [ ]:
###
# TASK: Increase the order until the fit is perfect
###

# -- FINISH THE CODE BELOW -----
(
    df_ex2
    >> ft_regression(
        var=["x"], 
        out=["y"], 
        order=2 # TODO: Increase the order until the fit is perfect
    )
    >> gr.ev_df(df=gr.df_make(x=np.linspace(-1, +1, num=100)))
    
    >> pt.ggplot(pt.aes("x", "y_mean"))
    + pt.geom_line(linetype="dashed")
    + pt.geom_point(
        data=df_ex2,
        mapping=pt.aes(y="y"),
    )
)

To help illustrate, let's look at one more synthetic example

In [ ]:
## NOTE: No need to edit
# More complicated function with noise
df_ex3 = (
    gr.df_make(x=np.linspace(-1, +1, num=40))
    >> gr.tf_mutate(y=fcn_2_noisy(DF.x))
)

(
    df_ex3
    >> pt.ggplot(pt.aes("x", "y"))
    + pt.geom_point()
)

Here we can see a somewhat complicated function that is quite corrupted by noise. Below, I'm going to fit a number of polynomial models of different orders. In practice, we would like to _make a decision_ about what polynomial order to use. A sensible choice would be to pick the order that minimizes the error -- let's see which model accomplishes this.

In [ ]:
# -- DEMONSTRATION CODE, NO NEED TO EDIT -----

# First, define a helper function to automate fitting a
# regression of user-selected `order`


def fit_poly(X, Y, order):
    poly = PolynomialFeatures(order)
    X_poly = poly.fit_transform(X)

    reg = LinearRegression(fit_intercept=False).fit(X_poly, Y)

    return reg, poly


# Setup to fit over different polynomial orders
Ord_all = list(range(15))
Ord_plot = [1, 3, 9]
Y_pred_c = np.zeros((len(Ord_all), X_c.shape[0]))
Y_pred_ref = np.zeros((len(Ord_all), X_ref.shape[0]))
NDE_c = np.zeros(len(Ord_all))
NDE_ref = np.zeros(len(Ord_all))

# Loop over orders
plt.figure()
plt.plot(X_ref, Y_2_ref, 'k--')
for i in range(len(Ord_all)):
    # Fit model
    reg, poly = fit_poly(X_c, Y_2_noisy, Ord_all[i])
    # Predict on same data & on reference points
    Y_pred_c[i] = reg.predict(poly.fit_transform(X_c))
    Y_pred_ref[i] = reg.predict(poly.fit_transform(X_ref))
    # Compute error *on same data* -> estimated error
    NDE_c[i] = nde(Y_2_noisy, Y_pred_c[i])
    # Compute error on reference points -> 'true' error
    NDE_ref[i] = nde(Y_2_ref, Y_pred_ref[i])

    # Plot curve
    if Ord_all[i] in Ord_plot:
        plt.plot(X_ref, Y_pred_ref[i], label="Order = {}".format(Ord_all[i]))
plt.plot(X_c, Y_2_noisy, 'k.')
plt.legend(loc=0)
plt.show()


Here I've selected just a few of the models to plot. We can see

* The `Order = 1` case is underfit, like we saw in the example above
* The `Order = 9` case curves tortuously to go through _every single point_; this is an example of _overfitting_
* The `Order = 3` case is not perfect, but fairly close to the true (dashed) curve. This is a well-fit model.

More generally, _overfitting_ is when the model fits to spurrious patterns in the data; essentially, we are fitting to noise, rather than signal. We would like to detect and avoid overfitting in practice! While we can see above some suspicious behavior based on the fitted curves, we might like a _quantitative_ way to compare models. We can do this with the NDE values, but there is a _subtle issue_ at play here.

Let's compare the `NDE` values `Estimated` on only the available (noisy) data, and the error computed using evaluations from the `True` (noiseless) function.

In [ ]:
plt.figure()
plt.plot(Ord_all, NDE_c, label='Estimated')
plt.plot(Ord_all, NDE_ref, label='True')
plt.legend(loc=0)
plt.xlabel('Poly Order')
plt.ylabel('ND Error')
plt.show()


Here we can see the `Estimated` and `True` error values _greatly diverge_. This is _highly problematic_ for two interrelated reasons:

1. In practice, we would only have access to the `Estimated` curve, as the `True` curve relies on data we do not have.
2. If we were to make a decision about `Poly Order` based on the `Estimated` curve, we would choose a much higher order than what would minimize the NDE in the `True` case.

The underlying reason for the poor error estimate here is that _we are using the same data to both train and test the model_. We can improve our estimates for the error through various techniques; below, we will use the technique of _cross-validation_.

### Avoiding Optimistic Estimates: Cross-Validation
[Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) is a technique for estimating the error in a way that avoids the "optimism" we saw above. For the variant _k-fold cross-validation_, we split all our data into _folds_, and use these to build _training_ and _test_ sets. Generally:

* _Training_ data are used to fit a model
* _Test_ data are used to evaluate a model

![CV schematic](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

(Fabian Flock, via Wikimedia)

In each of our $k$ iterations, we _do not allow_ the model to see a test fold (`Test data` above) during training, and fit the model only on the remaining data (`Training data` above). After training, we compute our chosen error metric on the test fold. Finally, we repeat this process on each of the $k$ chosen folds. This gives us a set of less optimistic estimates for the error, which we can summarize e.g. as a mean CV error.

This procedure is implemented in the scikit-learn function `cross_validate()`. Use this routine to estimate the NDE in the case where the polynomial order is `9`. You will have to choose a number of folds to run; while `5` to `10` is common, since we have so few data, you will need to use a smaller number of folds.

### Q5: K-fold Cross-Validation
Perform k-fold cross validation on the order `9` polynomial model using the scikit-learn function `cross_validate()`. You will need to look up the documentation for `cross_validate()`, and pick the `cv` strategy to use the k-fold strategy with a reasonable value for $k$. Make sure to report both the `train_score` and `test_score` values. Compare the two sets of values.

*Hints:*
- You can pass `nde_score` to the keyword argument `scoring` to have `cross_validate()` compute the NDE
- You can use the helper function `estimator, poly = fit_poly()` function above to help complete this task

In [ ]:
###
# TASK: Use cross_validate() on the 9th order model to estimate the NDE
# TODO: Compute the cross validation scores and assign to `scores`
# Hint, you can pass the helper function `nde_score` (defined above) to cross_validate()
###

# -- WRITE YOUR CODE HERE -----


# -- NO NEED TO EDIT BELOW -----
scores


The `train_score` values are quite optimistic, while the `test_score` values are *abysmal*.

Below, I show results for performing k-fold cross validation across the same set of polynomial orders as above.

In [ ]:
# REDACTED....
# -- NO NEED TO WRITE CODE HERE -----



<img src="../images/05_cv_order.png">
<!-- task-end -->

The individual CV estimates are reported as dots, while their mean is given as a line. Here we can see that the NDE takes reasonable values for order at or below `3`. Beyond this point, the NDE explodes as models begin to overfit wildly. These cross-validated error metrics would be far more informative for making a decision about polynomial order.

Here we have just one tunable knob (polynomial order) that defines our model. More generally, these kinds of user-selected quantities are called [hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)). Cross-validation and related techniques are key to _tuning hyperparameters_.

### Using Models for Materials Informatics
In the final part of the workshop, we will discuss how to _use_ these machine learning models to do useful work in materials science. Stay tuned!